In [1]:
conda install numpy

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c intel mkl

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yanni\anaconda3

  added / updated specs:
    - mkl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.10.8          |   py39haa95532_2         156 KB  intel
    openssl-1.1.1n             |       h2bbff1b_0         5.8 MB  intel
    ------------------------------------------------------------
                                           Total:         5.9 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                         pkgs/main --> intel
  openssl                                         pkgs/main --> intel




openssl-1.1.1n       | 5.8 MB    |            |   0% 
openssl-1.1.1n       | 5.8 MB    |            |   0% 
openssl-1.1.1n       | 5.8 MB    |            |   1% 
openssl-1.1.1n       | 5.8 MB    | 1          |   1% 


In [3]:
pip install -U numpy

  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.3 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.3 which is incompatible.


In [3]:
import pandas as pd 
import numpy as np 

from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.cluster import DBSCAN


import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe_connected"
from IPython.display import HTML, Image

In [4]:
df_apr = pd.read_csv('uber-raw-data-apr14.csv')

In [5]:
df_may = pd.read_csv('uber-raw-data-may14.csv')

In [6]:
df_jun = pd.read_csv('uber-raw-data-jun14.csv')

In [7]:
df_jul = pd.read_csv('uber-raw-data-jul14.csv')

In [8]:
df_aug = pd.read_csv('uber-raw-data-aug14.csv')

In [9]:
df_sep = pd.read_csv('uber-raw-data-sep14.csv')

In [10]:
df_def = pd.concat([df_apr, df_aug, df_jul, df_jun, df_may, df_sep])

In [11]:
df_def.columns = [x.lower().replace('/', '_') for x in df_def.columns]

In [12]:
df_def.date_time = pd.to_datetime(df_def['date_time'])

In [13]:
new_dates, new_times = zip(*[(d.date(), d.time()) for d in df_def['date_time']])
df_def = df_def.assign(new_date=new_dates, new_time=new_times)
df_def['hour'] = df_def.date_time.dt.hour
df_def['day'] = df_def.date_time.dt.dayofweek

In [14]:
df_def = df_def.drop(columns=["date_time"])

In [15]:
len(df_def)

4534327

DROP OUTLIERS

In [16]:
for i in ['lat', 'lon',]:
    df_def = df_def[~(np.abs(df_def[i]-df_def[i].mean()) > (3*df_def[i].std()))]

In [17]:
len(df_def)

4301626

VISUALISATION DES LUNDI TOUTES LES HEURES.

In [16]:
df_j1 = df_def[(df_def.day == 0) & (df_def.hour <=23)]

In [17]:
df_j1 = df_j1.drop(columns=['base', 'new_date', 'new_time'])

In [18]:
df_j1.head()

,lat,lon,hour,day
7785,40.7205,-73.9939,0,0
7786,40.7407,-74.0077,0,0
7787,40.7591,-73.9892,0,0
7788,40.7419,-74.0034,0,0
7789,40.7419,-74.0034,1,0


In [19]:
numeric_features = [0, 1, 2, 3] # Positions des colonnes quantitatives dans X
numeric_transformer = StandardScaler()

# On combine les transformers dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        
    ])



In [20]:
X_j1 = preprocessor.fit_transform(df_j1) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_j1)
np.unique(db.labels_)
df_j1["cluster"] = db.labels_

In [21]:
fig = px.scatter_mapbox(df_j1[(df_j1.cluster >=0)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour",
        title = "lundi_trafic_by_hour"
)
fig.show()

VISUALITION MARDI TOUTES LES HEURES

In [22]:
df_j2 = df_def[(df_def.day == 1) & (df_def.hour <=23)]

In [23]:
df_j2 = df_j2.drop(columns=['base', 'new_date', 'new_time'])

In [24]:
df_j2.head()

,lat,lon,hour,day
0,40.7690,-73.9549,0,1
1,40.7267,-74.0345,0,1
2,40.7316,-73.9873,0,1
3,40.7588,-73.9776,0,1
4,40.7594,-73.9722,0,1


In [25]:
numeric_features = [0, 1, 2, 3] # Positions des colonnes quantitatives dans X
numeric_transformer = StandardScaler()

# On combine les transformers dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        
    ])



In [26]:
X_j2 = preprocessor.fit_transform(df_j2) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_j2)
np.unique(db.labels_)
df_j2["cluster"] = db.labels_

In [27]:
fig = px.scatter_mapbox(df_j2[(df_j2.cluster >=0)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour",
        title = "mardi_trafic_by_hour"

)
fig.show()

VISUALITION MERCREDI TOUTES LES HEURES

In [23]:
df_j3 = df_def[(df_def.day == 2) & (df_def.hour <=23)].drop(columns=['base', 'new_date', 'new_time'])

In [24]:
numeric_features = [0, 1, 2, 3] # Positions des colonnes quantitatives dans X
numeric_transformer = StandardScaler()

# On combine les transformers dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        
    ])



In [25]:
X_j3 = preprocessor.fit_transform(df_j3) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_j3)
np.unique(db.labels_)
df_j3["cluster"] = db.labels_

In [26]:
fig = px.scatter_mapbox(df_j3[(df_j3.cluster >=0)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour",
        title = "mercredi_trafic_by_hour"

)
fig.show()

JEUDI

In [ ]:
df_j4 = df_def[(df_def.day == 3) & (df_def.hour <=23)].drop(columns=['base', 'new_date', 'new_time'])

In [ ]:
X_j4 = preprocessor.fit_transform(df_j4) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_j4)
np.unique(db.labels_)
df_j4["cluster"] = db.labels_

In [ ]:
fig = px.scatter_mapbox(df_j4[(df_j4.cluster >=0)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour",
        title = "jeudi_trafic_by_hour"

)
fig.show()

VENDREDI

In [ ]:
df_j5 = df_def[(df_def.day == 4) & (df_def.hour <=23)].drop(columns=['base', 'new_date', 'new_time'])

In [ ]:
X_j5 = preprocessor.fit_transform(df_j5) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_j5)
np.unique(db.labels_)
df_j5["cluster"] = db.labels_

In [ ]:
fig = px.scatter_mapbox(df_j5[(df_j5.cluster >=0)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour",
        title = "vendredi_trafic_by_hour"

)
fig.show()

SAMEDI

In [18]:
df_j6 = df_def[(df_def.day == 5) & (df_def.hour <=23)].drop(columns=['base', 'new_date', 'new_time'])

In [21]:
X_j6 = preprocessor.fit_transform(df_j6) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_j6)
np.unique(db.labels_)
df_j6["cluster"] = db.labels_

In [22]:
fig = px.scatter_mapbox(df_j6[(df_j6.cluster >=0)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour",
        title = "samedi_trafic_by_hour"

)
fig.show()

DIMANCHE

In [ ]:
df_j7 = df_def[(df_def.day == 6) & (df_def.hour <=23)].drop(columns=['base', 'new_date', 'new_time'])

In [ ]:
X_j7 = preprocessor.fit_transform(df_j7) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_j7)
np.unique(db.labels_)
df_j7["cluster"] = db.labels_

In [ ]:
fig = px.scatter_mapbox(df_j7[(df_j7.cluster >=0)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour",
        title = "dimanche_trafic_by_hour"

)
fig.show()